In [47]:
import geopandas as gpd
from shapely import Point
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import json
import yaml
from pyonemap import OneMap
from dotenv import load_dotenv
import requests
import app_utils as utils


In [49]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '../data')
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [52]:
def SP2_get_centroid_from_postal_code(address):
    load_dotenv()
    df = pd.read_csv(os.path.join(data_directory, 'Cluster_data','indiv_combined_centroid_data_fixed.csv'),index_col = 0)
    df = df[["centroid_name","Latitude_x","Longitude_x","MRT.Name"]]
    #search for average coords of api results
    one_map_email = os.getenv("ONE_MAP_EMAIL")
    one_map_password = os.getenv("ONE_MAP_PASSWORD")
    payload = {
            "email": one_map_email,
            "password": one_map_password
        }
    api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
    api_key = api_key.json()["access_token"]
    onemap = OneMap(api_key)
    location = onemap.search(address)
    if location['found'] and location['found'] > 0:
        lat, long =0,0
        for i in range(location['found']):
            lat += float(location['results'][i]['LATITUDE'])
            long += float(location['results'][i]['LONGITUDE'])
        lat = lat/location['found']
        long = long/location['found']
    else:
        print("No locations found")
        return None
    print(lat,long)
    #find nearest centroid
    df['euclidean'] = df.apply(lambda x: haversine(lat, long, x['Latitude_x'], x['Longitude_x']), axis=1)
    df = df[df['euclidean'] == df['euclidean'].min()]
    return (df["Latitude_x"].values[0], df["Longitude_x"].values[0])

In [53]:
SP2_get_centroid_from_postal_code("26 JALAN MEMBINA")

1.28263437649669 103.825595753657


(1.282626931, 103.826023)